In [1]:
# Importing Libraries

import time
import os
from datetime import datetime
from secedgar.filings import Filing, FilingType
from secedgar.cik_lookup import get_cik_map
time.sleep(10)
# os.getcwd()

import bs4 as bs
import requests
import pandas as pd
import re

In [2]:
# To avoid 'user limit exceeded'

import requests_random_user_agent

s = requests.Session()
print(s.headers['User-Agent'])

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36


In [3]:
cik = get_cik_map()['ticker']
print("Total number of companies are", len(cik))

# outputPath = "C:\\Users\\ARYAN\\Desktop\\9th Sem\\Project"

Total number of companies are 13023


In [4]:
#  Downloading 10-K files using SEC Library

'''
start = time.time()

total = 0

for idx, ck in enumerate(list(cik.keys())[:11]):
    print(idx, ck, cik[ck])
    
    temp = Filing(cik_lookup = list(cik.keys())[idx],
                filing_type = FilingType.FILING_10K,
                start_date = datetime(2016, 1, 1),
                end_date = datetime(2022, 1, 1),
                user_agent = 'aryanpcm@gmail.com')
    
    try:
        print("Filings found")
        temp.save(outputPath)
        
    except:
        total += 1
        print("No filings found")

end = time.time()
print('Total Time taken -----', round((end-start)/60, 2), 'minutes')'''

'\nstart = time.time()\n\ntotal = 0\n\nfor idx, ck in enumerate(list(cik.keys())[:11]):\n    print(idx, ck, cik[ck])\n    \n    temp = Filing(cik_lookup = list(cik.keys())[idx],\n                filing_type = FilingType.FILING_10K,\n                start_date = datetime(2016, 1, 1),\n                end_date = datetime(2022, 1, 1),\n                user_agent = \'aryanpcm@gmail.com\')\n    \n    try:\n        print("Filings found")\n        temp.save(outputPath)\n        \n    except:\n        total += 1\n        print("No filings found")\n\nend = time.time()\nprint(\'Total Time taken -----\', round((end-start)/60, 2), \'minutes\')'

In [5]:
list_of_companies = cik.keys()
lookup_table = cik

In [6]:
# Checking the content of cik
cik

{'AAPL': '320193',
 'MSFT': '789019',
 'GOOG': '1652044',
 'AMZN': '1018724',
 'GE': '40545',
 'TSLA': '1318605',
 'BRK-A': '1067983',
 'TSM': '1046179',
 'TCEHY': '1293451',
 'V': '1403161',
 'BABA': '1577552',
 'JPM': '19617',
 'JNJ': '200406',
 'WMT': '104169',
 'LVMUY': '824046',
 'SPY': '884394',
 'UNH': '731766',
 'MA': '1141391',
 'BAC': '70858',
 'HD': '354950',
 'PG': '80424',
 'ASML': '937966',
 'DIS': '1744489',
 'PYPL': '1633917',
 'ADBE': '796343',
 'PFE': '78003',
 'CMCSA': '1166691',
 'NKE': '320187',
 'LLY': '59478',
 'ORCL': '1341439',
 'TM': '1094517',
 'KO': '21344',
 'XOM': '34088',
 'NVO': '353278',
 'CSCO': '858877',
 'IDCBY': '1445468',
 'CRM': '1108524',
 'VZ': '732712',
 'NFLX': '1065280',
 'DHR': '313616',
 'ABT': '1800',
 'PEP': '77476',
 'INTC': '50863',
 'CHDRY': '1450488',
 'TMO': '97745',
 'NVS': '1114448',
 'ABBV': '1551152',
 'ACN': '1467373',
 'CIHKY': '1495688',
 'WFC': '72971',
 'T': '732717',
 'AVGO': '1730168',
 'COST': '909832',
 'MRK': '310158',


In [7]:
# Extracting relevant links

year = 2019
quarter = 'QTR2'

#get name of all filings 
download = requests.get(f'https://www.sec.gov/Archives/edgar/full-index/{year}/{quarter}/master.idx').content
download = download.decode("utf-8").split('\n')

In [8]:
# Checking Content of download
print(download[2001:2011])

['1007499|IFS SECURITIES, INC|X-17A-5|2019-06-14|edgar/data/1007499/9999999997-19-005663.txt', '1007524|OSTERWEIS CAPITAL MANAGEMENT INC|13F-HR|2019-05-15|edgar/data/1007524/0001085146-19-001576.txt', '1007546|BATINOVICH ANDREW|4|2019-05-06|edgar/data/1007546/0001209191-19-027760.txt', '1007547|KLAUSMEYER KEVIN|4|2019-06-24|edgar/data/1007547/0001567619-19-013912.txt', '1007549|SOLCHER STEPHEN B|4|2019-04-25|edgar/data/1007549/0001593968-19-000988.txt', '1007549|SOLCHER STEPHEN B|4|2019-05-08|edgar/data/1007549/0001593968-19-001070.txt', '1007549|SOLCHER STEPHEN B|4|2019-05-24|edgar/data/1007549/0001593968-19-001207.txt', '1007549|SOLCHER STEPHEN B|4|2019-06-26|edgar/data/1007549/0001593968-19-001498.txt', '1007571|RIVERSOURCE OF NEW YORK VARIABLE ANNUITY ACCOUNT|485BPOS|2019-04-25|edgar/data/1007571/0001193125-19-119265.txt', '1007571|RIVERSOURCE OF NEW YORK VARIABLE ANNUITY ACCOUNT|485BPOS|2019-04-25|edgar/data/1007571/0001193125-19-119291.txt']


In [9]:
# build the first part of the url

filing = '10-K'
list_of_links = []

for item in download:
    if(filing in item): 
        #print('Found')
        #print(item)
        company = item
        company = company.strip()
        splitted_company = company.split('|')
        url = splitted_company[-1]
        list_of_links.append(url)
        
print('Total number of links captured are', len(list_of_links))

Total number of links captured are 1988


In [10]:
# Sample URL creation

url = list_of_links[5]
print(url)

url2 = url.split('-') 
url2 = url2[0] + url2[1] + url2[2]
url2 = url2.split('.txt')[0]
print(url2) #edgar/data/1326801/000132680120000076

to_get_html_site = 'https://www.sec.gov/Archives/' + url
data = requests.get(to_get_html_site).content
data = data.decode("utf-8") 
data = data.split('FILENAME>')
# print(data[1])
data = data[1].split('\n')[0]

url_to_use = 'https://www.sec.gov/Archives/'+ url2 + '/' + data
print(url_to_use)  # Returns a link like https://www.sec.gov/Archives/edgar/data/1001601/000149315219005484/form10-k.htm

edgar/data/1001601/0001493152-19-005484.txt
edgar/data/1001601/000149315219005484
https://www.sec.gov/Archives/edgar/data/1001601/000149315219005484/form10-k.htm


In [11]:
# Capturing the complete URL

url_list = []

for i in range(len(list_of_links)):
    print('Starting for', i)
    url = list_of_links[i]
    print(url) #edgar/data/1326801/0001326801-20-000076.txt

    url2 = url.split('-') 
    url2 = url2[0] + url2[1] + url2[2]
    url2 = url2.split('.txt')[0]
    #print(url2) #edgar/data/1326801/000132680120000076

    to_get_html_site = 'https://www.sec.gov/Archives/' + url
    data = requests.get(to_get_html_site).content
    data = data.decode("utf-8") 
    data = data.split('FILENAME>')
    #print(data[1])
    data = data[1].split('\n')[0]
    url_to_use = 'https://www.sec.gov/Archives/'+ url2 + '/'+data
    #print(url_to_use)
    url_list.append(url_to_use)
    
print(len(url_list))

Starting for 0
edgar/data/1000045/0001564590-19-023956.txt
Starting for 1
edgar/data/1000683/0001213900-19-005351.txt
Starting for 2
edgar/data/1000694/0001144204-19-022376.txt
Starting for 3
edgar/data/1001463/0001185185-19-000474.txt
Starting for 4
edgar/data/1001601/0001493152-19-005718.txt
Starting for 5
edgar/data/1001601/0001493152-19-005484.txt
Starting for 6
edgar/data/1001601/0001493152-19-004537.txt
Starting for 7
edgar/data/1002047/0001564590-19-022779.txt
Starting for 8
edgar/data/1002135/0001002135-19-000017.txt
Starting for 9
edgar/data/1004434/0001004434-19-000011.txt
Starting for 10
edgar/data/1004724/0001564590-19-019674.txt
Starting for 11
edgar/data/1004724/0001564590-19-010589.txt
Starting for 12
edgar/data/1004980/0000950157-19-000501.txt
Starting for 13
edgar/data/1004989/0001437749-19-007807.txt
Starting for 14
edgar/data/1004989/0001437749-19-007194.txt
Starting for 15
edgar/data/1004989/0001437749-19-006183.txt
Starting for 16
edgar/data/1005011/0001005011-19-0

IndexError: list index out of range

In [12]:
# No specific point of termination
print('Loop breaking at', i)

Loop breaking at 82


In [13]:
# URL index at which loop terminated

url = list_of_links[i]
print(url)

url2 = url.split('-') 
url2 = url2[0] + url2[1] + url2[2]
url2 = url2.split('.txt')[0]
print(url2) #edgar/data/1326801/000132680120000076

to_get_html_site = 'https://www.sec.gov/Archives/' + url
data = requests.get(to_get_html_site).content
data = data.decode("utf-8") 
data = data.split('FILENAME>')
# print(data[1])
data = data[1].split('\n')[0]

url_to_use = 'https://www.sec.gov/Archives/'+ url2 + '/' + data
print(url_to_use)  # Returns a link like https://www.sec.gov/Archives/edgar/data/1001601/000149315219005484/form10-k.htm

edgar/data/1040850/0001554795-19-000115.txt
edgar/data/1040850/000155479519000115
https://www.sec.gov/Archives/edgar/data/1040850/000155479519000115/agtk0503form10k.htm


In [15]:
# Inspecting data for ith index
print(data)

agtk0503form10k.htm
